In this project, we will focus on healthcare. This data set is made available by MIT. It contains data about 9,026 heartbeat measurements. Each row represents a single measurement (captured on a timeline). There are a total of 80 data points (columns). This is a multiclass classification task: predict whether the measurement represents a normal heartbeat or other anomalies. 

## Description of Variables

You will use the **hearbeat_cleaned.csv** data set for this assignment. Each row represents a single measurement. Columns labeled as T1 from T80 are the time steps on the timeline (there are 80 time steps, each time step has only one measurement). 

The last column is the target variable. It shows the label (category) of the measurement as follows:<br>
0 = Normal<br>
1 = Supraventricular premature beat<br>
2 = Premature ventricular contraction<br>
3 = Fusion of ventricular and normal beat<br>
4 = Unclassifiable beat

## Goal

Use the data set **hearbeat_cleaned.csv** to predict the column called **Target**. The input variables are columns labeled as **T1 to T80**. 

# Note:

The data is cleaned up. There are no unqueal length sequences. And, there is no zero padding. So, you shouldn't use any `Masking` layer (like I mentioned in the lecture). 

In [3]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [2]:
#Get the data
heartbeat = pd.read_csv("heartbeat_cleaned.csv")
heartbeat.head()

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T72,T73,T74,T75,T76,T77,T78,T79,T80,Target
0,0.987,0.892,0.461,0.1130,0.1490,0.1900,0.1650,0.1620,0.1470,0.1380,...,0.1970,0.1970,0.1960,0.2030,0.201,0.1990,0.2010,0.205,0.2080,0
1,1.000,0.918,0.621,0.1330,0.1050,0.1250,0.1170,0.0898,0.0703,0.0781,...,0.1950,0.1910,0.1520,0.1720,0.207,0.2110,0.2070,0.207,0.1720,0
2,1.000,0.751,0.143,0.1040,0.0961,0.0519,0.0442,0.0416,0.0364,0.0857,...,0.2260,0.2420,0.2440,0.2860,0.468,0.8160,0.9770,0.452,0.0519,0
3,1.000,0.740,0.235,0.0464,0.0722,0.0567,0.0103,0.0155,0.0284,0.0155,...,0.0851,0.0747,0.0515,0.0593,0.067,0.0361,0.1210,0.451,0.8690,0
4,1.000,0.833,0.309,0.0191,0.1010,0.1200,0.1040,0.0874,0.0765,0.0765,...,0.2050,0.4210,0.8030,0.9510,0.467,0.0000,0.0519,0.082,0.0628,0


In [5]:
heartbeat.shape

(7960, 81)

In [7]:
y = heartbeat['Target']
x = heartbeat.drop('Target', axis=1)

In [8]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

In [9]:
#Target variables need to be an array with integer type
train_y = np.array(train_y)
test_y = np.array(test_y)

train_y = train_y.astype(np.int32)
test_y = test_y.astype(np.int32)


In [10]:
#Check the first 10 values of the train_y data set
train_y[0:10]

array([1, 0, 2, 0, 2, 2, 0, 0, 4, 4])

In [11]:
#Convert input variables to a 2-D array with float data type
train_x= np.array(train_x)
test_x= np.array(test_x)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [12]:
train_x

array([[1.    , 0.818 , 0.133 , ..., 0.354 , 0.354 , 0.387 ],
       [1.    , 0.849 , 0.166 , ..., 0.108 , 0.0811, 0.0695],
       [0.    , 0.0335, 0.163 , ..., 0.62  , 0.624 , 0.606 ],
       ...,
       [1.    , 0.98  , 0.573 , ..., 0.057 , 0.038 , 0.0418],
       [0.564 , 0.512 , 0.468 , ..., 0.357 , 0.348 , 0.354 ],
       [0.799 , 0.683 , 0.564 , ..., 0.265 , 0.265 , 0.247 ]],
      dtype=float32)

In [13]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))


In [14]:
train_x.shape, train_y.shape

((5572, 80, 1), (5572,))

In [15]:
train_x

array([[[1.    ],
        [0.818 ],
        [0.133 ],
        ...,
        [0.354 ],
        [0.354 ],
        [0.387 ]],

       [[1.    ],
        [0.849 ],
        [0.166 ],
        ...,
        [0.108 ],
        [0.0811],
        [0.0695]],

       [[0.    ],
        [0.0335],
        [0.163 ],
        ...,
        [0.62  ],
        [0.624 ],
        [0.606 ]],

       ...,

       [[1.    ],
        [0.98  ],
        [0.573 ],
        ...,
        [0.057 ],
        [0.038 ],
        [0.0418]],

       [[0.564 ],
        [0.512 ],
        [0.468 ],
        ...,
        [0.357 ],
        [0.348 ],
        [0.354 ]],

       [[0.799 ],
        [0.683 ],
        [0.564 ],
        ...,
        [0.265 ],
        [0.265 ],
        [0.247 ]]], dtype=float32)

Baseline 

In [16]:
heartbeat['Target'].value_counts()/len(heartbeat)

0    0.582035
4    0.198995
2    0.155402
1    0.055905
3    0.007663
Name: Target, dtype: float64

#### Build a cross-sectional shallow model using Keras (with only one hidden layer)

In [17]:
n_steps = 80
n_inputs = 1


model = keras.models.Sequential([
    
    keras.layers.SimpleRNN(5, activation='softmax' , input_shape=[n_steps, n_inputs]),
    
])

In [18]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

In [21]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)


model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=80,
                    validation_data=(test_x, test_y), callbacks=callback)

Epoch 1/80
175/175 [==============================] - 3s 14ms/step - loss: 1.2218 - accuracy: 0.5774 - val_loss: 1.1084 - val_accuracy: 0.5930
Epoch 2/80
175/175 [==============================] - 2s 12ms/step - loss: 1.1199 - accuracy: 0.5774 - val_loss: 1.0942 - val_accuracy: 0.5930
Epoch 3/80
175/175 [==============================] - 2s 13ms/step - loss: 1.1111 - accuracy: 0.5635 - val_loss: 1.0860 - val_accuracy: 0.5775
Epoch 4/80
175/175 [==============================] - 2s 11ms/step - loss: 1.1083 - accuracy: 0.5628 - val_loss: 1.0846 - val_accuracy: 0.5611
Epoch 5/80
175/175 [==============================] - 2s 11ms/step - loss: 1.1079 - accuracy: 0.5528 - val_loss: 1.0967 - val_accuracy: 0.5389
Epoch 6/80
175/175 [==============================] - 2s 11ms/step - loss: 1.1073 - accuracy: 0.5472 - val_loss: 1.0828 - val_accuracy: 0.5812
Epoch 7/80
175/175 [==============================] - 2s 11ms/step - loss: 1.1063 - accuracy: 0.5499 - val_loss: 1.0821 - val_accuracy: 0.5465

In [22]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores


[1.060469627380371, 0.6143215894699097]

In [23]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 1.06
accuracy: 61.43%


#### Build a cross-sectional deep model using Keras (with two or more hidden layers)

In [24]:
n_steps = 80
n_inputs = 1


model = keras.models.Sequential([
    keras.layers.SimpleRNN(10, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(12, return_sequences=True),
    keras.layers.SimpleRNN(13), 
    keras.layers.Dense(5, activation='softmax')
])


In [25]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)


model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(train_x, train_y, epochs=40,
                    validation_data=(test_x, test_y), callbacks=callback)

Epoch 1/40
175/175 [==============================] - 9s 40ms/step - loss: 1.1151 - accuracy: 0.5831 - val_loss: 1.1062 - val_accuracy: 0.5930
Epoch 2/40
175/175 [==============================] - 6s 37ms/step - loss: 1.1406 - accuracy: 0.5774 - val_loss: 1.1588 - val_accuracy: 0.5930
Epoch 3/40
175/175 [==============================] - 6s 32ms/step - loss: 1.1366 - accuracy: 0.5770 - val_loss: 1.1420 - val_accuracy: 0.5930
Epoch 4/40
175/175 [==============================] - 7s 39ms/step - loss: 1.0989 - accuracy: 0.5766 - val_loss: 1.0743 - val_accuracy: 0.5745
Epoch 5/40
175/175 [==============================] - 6s 36ms/step - loss: 1.1104 - accuracy: 0.5727 - val_loss: 1.1265 - val_accuracy: 0.4116
Epoch 6/40
175/175 [==============================] - 6s 35ms/step - loss: 1.0870 - accuracy: 0.5799 - val_loss: 1.0807 - val_accuracy: 0.5930
Epoch 7/40
175/175 [==============================] - 6s 35ms/step - loss: 1.1355 - accuracy: 0.5774 - val_loss: 1.1538 - val_accuracy: 0.5930

In [26]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores



[1.1106446981430054, 0.5929648280143738]

In [27]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 1.11
accuracy: 59.30%


#### Build a sequential shallow LSTM Model (with only one LSTM layer)

In [28]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(10, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(5, activation='softmax')
])
    
   

In [31]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=50,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/50
175/175 [==============================] - 7s 28ms/step - loss: 1.1081 - accuracy: 0.5734 - val_loss: 1.0208 - val_accuracy: 0.5930
Epoch 2/50
175/175 [==============================] - 4s 25ms/step - loss: 1.0571 - accuracy: 0.6053 - val_loss: 1.0612 - val_accuracy: 0.6353
Epoch 3/50
175/175 [==============================] - 4s 25ms/step - loss: 1.0704 - accuracy: 0.5917 - val_loss: 1.1310 - val_accuracy: 0.5930
Epoch 4/50
175/175 [==============================] - 4s 25ms/step - loss: 1.0493 - accuracy: 0.5774 - val_loss: 1.0784 - val_accuracy: 0.5930
Epoch 5/50
175/175 [==============================] - 5s 26ms/step - loss: 1.0346 - accuracy: 0.5870 - val_loss: 0.9534 - val_accuracy: 0.6390
Epoch 6/50
175/175 [==============================] - 5s 26ms/step - loss: 0.9269 - accuracy: 0.6716 - val_loss: 0.9911 - val_accuracy: 0.6110
Epoch 7/50
175/175 [==============================] - 4s 26ms/step - loss: 0.7191 - accuracy: 0.7633 - val_loss: 0.7211 - val_accuracy: 0.7596

In [34]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

[0.49611639976501465, 0.837520956993103]

In [35]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.50
accuracy: 83.75%


#### Build a sequential deep LSTM Model (with only two LSTM layers)

In [36]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(6, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(7),
    keras.layers.Dense(5, activation='softmax')
])

In [38]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=30,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/30
175/175 [==============================] - 12s 52ms/step - loss: 1.1062 - accuracy: 0.5931 - val_loss: 1.0283 - val_accuracy: 0.6386
Epoch 2/30
175/175 [==============================] - 9s 49ms/step - loss: 1.0560 - accuracy: 0.6181 - val_loss: 1.0450 - val_accuracy: 0.6353
Epoch 3/30
175/175 [==============================] - 9s 49ms/step - loss: 0.9724 - accuracy: 0.6522 - val_loss: 0.8807 - val_accuracy: 0.6951
Epoch 4/30
175/175 [==============================] - 9s 52ms/step - loss: 0.8116 - accuracy: 0.7037 - val_loss: 0.8329 - val_accuracy: 0.7111
Epoch 5/30
175/175 [==============================] - 9s 52ms/step - loss: 0.7597 - accuracy: 0.7376 - val_loss: 0.7094 - val_accuracy: 0.7730
Epoch 6/30
175/175 [==============================] - 9s 49ms/step - loss: 0.7128 - accuracy: 0.7514 - val_loss: 0.6812 - val_accuracy: 0.7747
Epoch 7/30
175/175 [==============================] - 9s 51ms/step - loss: 0.6993 - accuracy: 0.7543 - val_loss: 0.7360 - val_accuracy: 0.734

In [40]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

[0.731451153755188, 0.7495812177658081]

In [41]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.73
accuracy: 74.96%


#### Build a sequential deep LSTM Model (with only two LSTM layers)

In [42]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(6, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(5, activation='softmax')
])

In [44]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=20,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/20
175/175 [==============================] - 7s 28ms/step - loss: 1.1187 - accuracy: 0.5777 - val_loss: 1.0185 - val_accuracy: 0.5959
Epoch 2/20
175/175 [==============================] - 5s 28ms/step - loss: 1.0345 - accuracy: 0.5942 - val_loss: 1.0146 - val_accuracy: 0.5766
Epoch 3/20
175/175 [==============================] - 5s 31ms/step - loss: 1.0190 - accuracy: 0.6027 - val_loss: 1.0369 - val_accuracy: 0.6235
Epoch 4/20
175/175 [==============================] - 5s 26ms/step - loss: 1.0012 - accuracy: 0.6127 - val_loss: 0.9761 - val_accuracy: 0.6319
Epoch 5/20
175/175 [==============================] - 5s 27ms/step - loss: 0.9889 - accuracy: 0.6262 - val_loss: 0.9834 - val_accuracy: 0.5992
Epoch 6/20
175/175 [==============================] - 5s 27ms/step - loss: 0.9641 - accuracy: 0.6307 - val_loss: 1.0048 - val_accuracy: 0.6453
Epoch 7/20
175/175 [==============================] - 5s 29ms/step - loss: 0.8788 - accuracy: 0.6784 - val_loss: 0.8608 - val_accuracy: 0.6847

In [45]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.6117503046989441, 0.7973199486732483]

In [46]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.61
accuracy: 79.73%


#### Build a sequential deep GRU Model (with only two GRU layers) 

In [51]:
n_steps = 80
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(6, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(8),
    keras.layers.Dense(5, activation='softmax')
])

In [52]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=30,
                   validation_data = (test_x, test_y), callbacks=callback)

Epoch 1/30
175/175 [==============================] - 13s 56ms/step - loss: 1.1036 - accuracy: 0.5844 - val_loss: 1.0027 - val_accuracy: 0.6089
Epoch 2/30
175/175 [==============================] - 9s 50ms/step - loss: 0.9592 - accuracy: 0.6287 - val_loss: 0.8497 - val_accuracy: 0.6566
Epoch 3/30
175/175 [==============================] - 9s 50ms/step - loss: 0.7134 - accuracy: 0.7416 - val_loss: 0.8896 - val_accuracy: 0.6692
Epoch 4/30
175/175 [==============================] - 9s 49ms/step - loss: 0.5699 - accuracy: 0.8214 - val_loss: 0.5192 - val_accuracy: 0.8472
Epoch 5/30
175/175 [==============================] - 8s 48ms/step - loss: 0.5413 - accuracy: 0.8333 - val_loss: 0.4966 - val_accuracy: 0.8522
Epoch 6/30
175/175 [==============================] - 9s 49ms/step - loss: 0.5643 - accuracy: 0.8191 - val_loss: 0.5630 - val_accuracy: 0.8342
Epoch 7/30
175/175 [==============================] - 9s 51ms/step - loss: 0.5023 - accuracy: 0.8460 - val_loss: 0.5225 - val_accuracy: 0.825

In [53]:
# evaluate the model

scores = model.evaluate(test_x, test_y, verbose=0)

scores



[0.31149041652679443, 0.9091289639472961]

In [54]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.31
accuracy: 90.91%


# Discussion

#### Test values of each model 

#### Model performance